In [1]:
## This notebook ingest the raw "pit_stops" oracle redbull file from the bronze volume to bronze table
### Ingest pit_stops.csv file

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
bronze_folder_path    = "/Volumes/f1_bronze/bronze/f1_bronze_volume"
bronze_catalog    = "f1_bronze"
bronze_schema     = "bronze"
bronze_table_dlt = "f1_pit_stops_dlt"
bronze_table_par = "f1_pit_stops_par"


# ----------
##### Step 1 - Read the CSV file using the spark dataframe reader
#  ----------

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

pit_stops_schema = StructType([
    StructField("RACEID", IntegerType(), True),
    StructField("DRIVERID", IntegerType(), True),
    StructField("STOP", IntegerType(), True),
    StructField("LAP", IntegerType(), True),
    StructField("TIME", StringType(), True),                 # Format: HH:mm:ss
    StructField("DURATION", DoubleType(), True),             # Seconds as decimal
    StructField("MILLISECONDS", IntegerType(), True)
])

In [1]:
pitstops_df = spark.read \
.option("header", True) \
.schema(pit_stops_schema) \
.csv(f"{bronze_folder_path}/pit_stops.csv")

# ----------
##### Step 2 - Write the output to bronze catalog as parquet file or delta/parquet table
#  ----------

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        pitstops_df.write.mode("overwrite").parquet(f"{bronze_folder_path}/pit_stops_par")
elif target_type == 'table':
    if target_format == 'parquet':
        pitstops_df.write.mode("overwrite").format("parquet").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        pitstops_df.write.mode("overwrite").format("delta").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        pitstops_read_df = spark.read.parquet(f"{bronze_folder_path}/pit_stops_par")
elif target_type == 'table':
    if target_format == 'parquet':
        pitstops_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        pitstops_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")